# 11 lines of no-package Python code that solve Matt Parker's Wordle challenge in 8 seconds - and 18 lines with numpy that do so in 4

(c) by Thomas Reichert

## License

This program is free software; you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation; either version 3 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.
You should have received a copy of the GNU General Public License
along with this program; if not, see http://www.gnu.org/licenses/.

## The challenge and my code

The goal of this jupyter notebook is to solve [Matt Parker's](https://standupmaths.com) Wordle challenge (see [his youtube video](https://www.youtube.com/watch?v=c33AZBnRHks&t=959s)) to find combinations of five english five letter words that contain 25 distinct letters **with my personal restriction that all has to happen in not significantly more than 10 lines of basic Python code using only one single thread and no additional packages that might speed up things**.

The motivation for this is that if one almost exclusively works with Python such as yours truly as a data scientist and certified actuary does, the fact that solving this problem is possible much faster by just implementing it in e.g. C++ doesn't really help a lot as one could not spontaneusly write code in it if one hasn't worked with it yet. Also, I want to showcase that - despite clearly not being the fastest language around - Python is a great choice to solve small problems like this one with *just a few humanly understandable lines of code* - given that one understands how list comprehensions work ;-) So Python it is with currently 12 lines that each fully fit into a Jupyter cell's default width and that in reality are 11 as tqdm is only imported and used to make the user feel better because 'something happens on the screen all the time'.

*If you run this notebook, please run the code itself first and patiently wait for about 11 seconds as only then the values in the next markdown section will appear properly*

In [1]:
from tqdm import tqdm # only used to feel better because 'something happens on the screen all the time' ;-)
def cmb(x): # function that combines words recursively to tuples, triples, quadruples and quintuples
    return cmb([{'_'.join([kx, ky]): vx|vy for kx, vx in tqdm(x.pop(0).items()) for ky, vy in x[0].items()
                 if (vx&vy==0)}] + x[1:]) if (len(x) > 1) else x[0]
def val(f='words_alpha.txt', ana=True): # ana = True drops anagrams which speeds up code by factor 2
    tmp = {w: sum(1<<(ord(c)-97) for c in w) for w in open(f).read().split('\n') if (len(w)==len(set(w))==5)}
    return {v: k for k, v in {v: k for k, v in tmp.items()}.items()} if ana else tmp
x = val(); valid = {str(n): {k: v for k, v in x.items() if (len(set(k) & set('aeiou'))<=n)} for n in range(4)}
valid.update({i: {k: v for k, v in valid['1'].items() if len(set(k) & set(j))>0}
              for i, j in {'a':'a','e':'e','i':'i','o':'o','u':'u','x':'eou','y':'eiu','z':'aeu'}.items()})
result = [k for d in [cmb([valid[v] for v in i]) for i in ['ueoia', '0xyz2', '00133']] for k in d.keys()]
open('result.csv', 'w').write('\n'.join(result:=set(','.join(sorted(k.split('_'))) for k in result)))

100%|██████████| 4758/4758 [00:00<00:00, 6818.23it/s]


16139

## What the code actually does
As the runtime of this algorithm for finding all possible combinations of five words with all distinct letters will be $O(n_1 * n_2 * n_3 * n_4 * n_5)$, we must keep our numbers of words $n_i$ for the first, second, etc. word as small as only possible. Hence the first thing we must do is to get rid of as many words as possible beforehand and divide our problem into as small as only possible subgroups.

For this, we drop all anagrams, meaning that out of e.g. *below, bowel and elbow* we only keep one as all three contain the same five letters. Then we can calculate an integer representation of each word, where in the binary number system a = 1, b = 10, c = 100, etc. Comparing the integer numbers resulting from these binary numbers is about 7 times faster than creating letter sets and comparing those, whereas dropping anagrams is about twice as fast as keeping them.

We can also make use of the fact that there are only {{ len(valid.get('0')) }} valid words such as crypt, glyph or nymph in the word list that contain none of the five vowels a, e, i, o, or u. This means that a combination of five words can only contain a word with two vowels if at least one of them contains none. As be seen easily (almost all of these either contain either an x or a y), the maximum number of words from that vowelless list that can be combined is two.

So we solve the problem in three parts, each of them starting from the smallest to the largest list of words to keep the number of comparisons on the way as low as possible:

* Five words where each contains none or one vowel. Here we can even split the word list by vowel to speed things up using {{ len(valid.get('u')) }} words that contain u, {{ len(valid.get('e')) }} words that contain e, {{ len(valid.get('o')) }} words that contain o, {{ len(valid.get('i')) }} words that contain i and {{ len(valid.get('a')) }} words that contain a.
* One out of {{ len(valid.get('0')) }} words that contains no vowel, three out of {{ len(valid.get('1')) }} words that contain at most one and one out of {{ len(valid.get('2')) }} words that contains at most two vowels. As proven in the appendix, one does not have to check the full one-vowel-list three times, it is enough to check the {{ len(valid.get('x')) }} words that contain e, o or u first, then the {{ len(valid.get('y')) }} words that contain e, i or u and then the {{ len(valid.get('z')) }} words that contain a, e or u.
* Two out of {{ len(valid.get('0')) }} words that contain no vowel, one out of {{ len(valid.get('1')) }} words that contains at most one and two out of {{ len(valid.get('3')) }} words that contain at most three vowels.

and build everything together to find the {{ len(result) }} combinations without anagrams. While clearly being way off the results from the super fast precompiled languages, I was at least able to get a runtime of about 8 seconds on a M1 Macbook Pro and beat [Benjamin Paassen's graph theory approach](https://gitlab.com/bpaassen/five_clique), which was the first decently fast pure Python approach that I am aware of and served as my initial benchmark, by approximately a factor of more than 100 ;-) Meanwhile due to his comparison spreadsheet I had to figure out that there are already some Python solutions out there that are still faster than mine.

## Speeding up everything by allowing for vectorized numpy operations
Dropping the *no imported packages* constraint, I was able to speed up the code by roughly factor 2 using the well-known *numpy* package which allows for much faster vectorized operations instead of time costly for loops which are a known speed bottleneck in Python. This solution is conceptionally still the same algorithm, still single-threaded, below 20 lines of Python code and runs in 4 seconds. 

Will continue trying other optimization techniques - curious where the journey will take me ;-)

In [2]:
from tqdm import tqdm; import numpy as np
def npcmb(x):  # function that combines words recursively to tuples, triples, quadruples and quintuples
    if len(x) > 1:
        kx, vx, ky, vy = list(x[0].keys()), list(x[0].values()), list(x[1].keys()), list(x[1].values())
        tmp = np.where(np.bitwise_and(np.array(vx).repeat(len(vy)).reshape(len(vx), len(vy)),
                                      np.array(vy).repeat(len(vx)).reshape(len(vy), len(vx)).transpose())==0)
        tmp = [{'_'.join([kx[tmp[0][i]], ky[tmp[1][i]]]): vx[tmp[0][i]] | vy[tmp[1][i]]
                       for i in range(len(tmp[0]))}] + x[2:]
        return npcmb(tmp)
    return x[0]
def val(f='words_alpha.txt', ana=True): # ana = True drops anagrams which speeds up code by factor 2
    tmp = {w: sum(1<<(ord(c)-97) for c in w) for w in open(f).read().split('\n') if (len(w)==len(set(w))==5)}
    return {v: k for k, v in {v: k for k, v in tmp.items()}.items()} if ana else tmp
x = val(); valid = {str(n): {k: v for k, v in x.items() if (len(set(k) & set('aeiou'))<=n)} for n in range(4)}
valid.update({i: {k: v for k, v in valid['1'].items() if len(set(k) & set(j))>0}
              for i, j in {'a':'a','e':'e','i':'i','o':'o','u':'u','x':'eou','y':'eiu','z':'aeu'}.items()})
result = [k for d in [npcmb([valid[v] for v in i]) for i in ['ueoia', '0xyz2', '00133']] for k in d.keys()]
open('result.csv', 'w').write('\n'.join(result:=set(','.join(sorted(k.split('_'))) for k in result)))

16139

## The result
Here are the {{ len(result) }} word combinations without anagrams:

In [3]:
print(len(result))
print(sorted(result))

538
['ampyx,bejig,fconv,hdqrs,klutz', 'ampyx,bewig,fconv,hdqrs,klutz', 'ampyx,bortz,chivw,fjeld,gunks', 'ampyx,crwth,fdubs,kling,vejoz', 'ampyx,flung,hdqrs,twick,vejoz', 'avick,benjy,fldxt,grosz,whump', 'backs,fldxt,grump,vejoz,whiny', 'backs,fldxt,humpy,vejoz,wring', 'backs,fldxt,murph,vejoz,wingy', 'backs,fldxt,ringy,vejoz,whump', 'backs,fldxt,rumpy,vejoz,whing', 'backy,fldxt,grump,vejoz,whins', 'backy,fldxt,murph,vejoz,wings', 'backy,fldxt,rings,vejoz,whump', 'backy,fldxt,rumps,vejoz,whing', 'backy,fldxt,sumph,vejoz,wring', 'baken,chivw,fldxt,grosz,jumpy', 'bangs,fldxt,humpy,vejoz,wrick', 'bangs,fldxt,murph,vejoz,wicky', 'bangs,fldxt,ricky,vejoz,whump', 'bangs,fldxt,rumpy,vejoz,whick', 'bangy,crump,fldxt,vejoz,whisk', 'bangy,fldxt,murph,vejoz,wicks', 'bangy,fldxt,ricks,vejoz,whump', 'bangy,fldxt,rumps,vejoz,whick', 'bangy,fldxt,sumph,vejoz,wrick', 'bangy,flump,hdqrs,twick,vejoz', 'banks,fldxt,gyric,vejoz,whump', 'banky,crump,fldxt,vejoz,whigs', 'bargh,fldxt,numps,vejoz,wicky', 'bark

## Appendix
### Numbers of words that contain only the allowed vowels and no others by vowel or vowel triple
These numbers are important because one should start with the vowel or vowel triple that has the lowest number of words to keep the number of n-word combinations as small as possible when progressing n from 1 to 5.

Overall there are 5 vowels (which we will use all) and 10 possible vowel triples (of wich we will use only 3).

In [4]:
{i: len({k: v for k, v in valid['1'].items() if len(set(k) & set(i))>0})
 for i in ['a','e','i','o','u','aei','aeo','aeu','aio','aiu', 'aou','eio','eiu','eou','iou']}

{'a': 548,
 'e': 370,
 'i': 411,
 'o': 388,
 'u': 325,
 'aei': 1329,
 'aeo': 1306,
 'aeu': 1243,
 'aio': 1347,
 'aiu': 1284,
 'aou': 1261,
 'eio': 1169,
 'eiu': 1106,
 'eou': 1083,
 'iou': 1124}

### Proof that the 3 selected vowel triples allow all 10 vowel combinations of 3 one-vowel words

In [5]:
import itertools
len(sorted(list(set(''.join(sorted(i)) for i in list(
    itertools.product(*[list('eou'), list('eiu'), list('aeu')]))if len(set(i))==3))))

10

##  Final thoughts on my approach and my favourite solution by somebody else
### My own approach
Even though this approach clearly is no speed champ, my hope is that this *vowel based reduction of possible word combinations that need to be checked* can maybe help somebody else's super fast algorithm to even save a bit more time ;-)

What I found a surprise in the progress is that only a few out of these {{ len(result) }}  combinations contain none of those {{ len(valid.get('0')) }} non-vowel words. Unfortunately finding exactly those few combinations is currently the speed bottleneck of my approach. Originally I thought that combinations with non-vowel words would be the exception rather than the rule, but I had to learn that using one of them opens the chance for another one to contain one word with two vowels, which are {{ round(len(valid.get('2'))/len(valid.get('1')),1) }} times more frequent than words with only one vowel.

### Appreciation of other solutions
In the end, for me working as a data scientist coding is always a compromise between computational speed and the time and effort required for writing code or maintaining code written by somebody else. There are many situations where a blazing fast solution consisting of a lot of complex lines of code is optimal, but there alre also others where this code can be inferior to a handful lines of easily understandable and maintainable code despite the latter runs several times longer. That’s exactly what I love about Python — if speed at any cost is not your prime requirement you can be extremely code efficient. And if it is, it forces you to think about clever algorithm implementations instead of just relying on the intrinsic speed advantages of a programming language such as C++ ;-)

Thad said, I really like [Stefan Pochmann's](https://replit.com/@pochmann/5words538?v=1) solution for its combination of code brevity and runtime speed. Imho he solved this challenge way better than I did. Not only does his solution only take a third of my no-numpy solution's runtime, it is also the only other solution that I am aware of that not only came close to mine with respect to code brevity but even clearly beat me there, too.

Here's a slight adaption of Stefan's code, the only mentionable changes I made are that the solution now is actually returned as an object that could be processed further instead of just being printed out, and a switch for deciding whether one wants to drop anagrams beforehand or not. So all kudos for the algorithm itself should still go entirely to Stefan!

In [6]:
def solve(words, chset=set('abcdefghijklmnopqrstuvwxyz'), used=[], result=[], drop_ana=True):
    if not chset: return result + [[*(u for u in used if len(u)==5)]]
    if drop_ana: words = [*{frozenset(w): w for w in words}.values()]
    for w in words+(len(chset)%5)*[c:=min(chset, key=''.join(words).count)]:
        if c in w: result = solve([*filter({*w}.isdisjoint, words)], chset-{*w}, used+[w], result, False)
    return result
solution = solve([w for w in open('words_alpha.txt').read().split() if 5==len(w)==len(set(w))])

In [7]:
print(len(solution))

538
